In [1]:
!ls

Untitled.ipynb		    hmw5.ipynb		   zones
fhvhv_tripdata_2021-02.csv  taxi+_zone_lookup.csv


## Question1

In [11]:
from pyspark.sql import types
import pyspark
from pyspark.sql import SparkSession


In [12]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

### Answer1

In [22]:
spark.version

'3.0.3'

## Question2

In [13]:
schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True)
])

In [15]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhvhv_tripdata_2021-02.csv')

In [16]:
df = df.repartition(24)

In [17]:
df.write.parquet('fhvhv/2021/02/')

In [20]:
!ls

Untitled.ipynb	fhvhv_tripdata_2021-02.csv  taxi+_zone_lookup.csv
fhvhv		hmw5.ipynb		    zones


### Answer2

In [21]:
!du -h fhvhv/2021/

210M	fhvhv/2021/02
210M	fhvhv/2021/


## Question3

In [25]:
df = spark.read.parquet('fhvhv/2021/02/')


In [26]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)



In [31]:
df.count()

11613942

In [43]:
df.show(5)

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02887|2021-02-06 01:18:35|2021-02-06 01:40:34|         163|         235|   null|
|           HV0005|              B02510|2021-02-05 07:13:06|2021-02-05 07:31:56|         225|         181|   null|
|           HV0003|              B02869|2021-02-04 16:56:52|2021-02-04 17:21:36|         260|          95|   null|
|           HV0003|              B02871|2021-02-03 18:34:17|2021-02-03 18:57:12|         235|          60|   null|
|           HV0003|              B02869|2021-02-04 07:25:09|2021-02-04 07:30:34|          55|          55|   null|
+-----------------+--------------------+-------------------+-------------------+

In [36]:
df.registerTempTable('fhvhv_data')

### Answer3

In [50]:
spark.sql("""
SELECT
    pickup_datetime
FROM
    fhvhv_data
WHERE
    DATE(pickup_datetime) == '2021-02-15'

""").count()

367170

## Question 4

In [95]:
spark.sql("""
SELECT
    pickup_datetime, dropoff_datetime, (unix_timestamp(dropoff_datetime) - unix_timestamp(pickup_datetime)) AS DIFF
FROM
    fhvhv_data
ORDER BY
    (unix_timestamp(dropoff_datetime) - unix_timestamp(pickup_datetime) ) DESC

""").show(10)

+-------------------+-------------------+-----+
|    pickup_datetime|   dropoff_datetime| DIFF|
+-------------------+-------------------+-----+
|2021-02-11 13:40:44|2021-02-12 10:39:44|75540|
|2021-02-17 15:54:53|2021-02-18 07:48:34|57221|
|2021-02-20 12:08:15|2021-02-21 00:22:14|44039|
|2021-02-03 20:24:25|2021-02-04 07:41:58|40653|
|2021-02-19 23:17:44|2021-02-20 09:44:01|37577|
|2021-02-25 17:13:35|2021-02-26 02:57:05|35010|
|2021-02-20 01:36:13|2021-02-20 11:16:19|34806|
|2021-02-18 15:24:19|2021-02-19 01:01:11|34612|
|2021-02-18 01:31:20|2021-02-18 11:07:15|34555|
|2021-02-10 20:51:39|2021-02-11 06:21:08|34169|
+-------------------+-------------------+-----+
only showing top 10 rows



### Answer4

In [101]:
spark.sql("""
SELECT
    DATE(pickup_datetime) AS Longest
FROM
    fhvhv_data
ORDER BY
    (unix_timestamp(dropoff_datetime) - unix_timestamp(pickup_datetime) ) DESC

""").show(1)

+----------+
|   Longest|
+----------+
|2021-02-11|
+----------+
only showing top 1 row



## Question5

In [113]:
spark.sql("""
SELECT
    dispatching_base_num, COUNT(*) AS Count
FROM
    fhvhv_data
GROUP BY
    dispatching_base_num
ORDER BY
    Count DESC

""").show(5)

+--------------------+-------+
|dispatching_base_num|  Count|
+--------------------+-------+
|              B02510|3233664|
|              B02764| 965568|
|              B02872| 882689|
|              B02875| 685390|
|              B02765| 559768|
+--------------------+-------+
only showing top 5 rows



### Answer5

Check Spark page: 2 stages

## Question 6

In [ ]:
zones = spark.read \
    .option("header", "true") \
    .csv('taxi+_zone_lookup.csv')

zones.registerTempTable('zone_data')

In [157]:
spark.sql("""
SELECT
    CONCAT(COALESCE(zpu.Zone,'Unknown'), ' / ',  COALESCE(zdo.Zone,'Unknown') ) AS Pairs,
    count(1) as Count

FROM 
    fhvhv_data,
    zone_data zpu,
    zone_data zdo
    
WHERE
    fhvhv_data.PULocationID = zpu.LocationID AND  
        fhvhv_data.DOLocationID = zdo.LocationID 
GROUP BY
    Pairs
ORDER BY
    Count DESC


""").show(5)

+--------------------+-----+
|               Pairs|Count|
+--------------------+-----+
|East New York / E...|45041|
|Borough Park / Bo...|37329|
| Canarsie / Canarsie|28026|
|Crown Heights Nor...|25976|
|Bay Ridge / Bay R...|17934|
+--------------------+-----+
only showing top 5 rows



Answer6: East New York / East New York